In [1]:
# Should be numpy <2 for it to work with aims
!pip list | grep numpy 

numpy                         1.21.5
numpydoc                      1.2

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!source ~/.bashrc && pixi list | grep ipython 

/usr/bin/sh: 1: source: not found


In [3]:
from soma.qt_gui.qt_backend import Qt
from soma import aims
import anatomist.api as ana
import numpy as np
import matplotlib.pyplot as plt
import torch
%matplotlib qt5

In [4]:
from pathlib import Path
# Loading tmp npy data : 

root_tmp = Path("/neurospin/dico/tsanchez/tmp/")
argmax_inf = np.load(root_tmp / "output_inference.npy")
inputs = np.load(root_tmp / "input_inference.npy")
inputs.shape, np.unique(inputs), argmax_inf.shape, np.unique(argmax_inf)

((1, 60, 120, 144),
 array([-1.,  0.,  1.], dtype=float32),
 (1, 60, 120, 144),
 array([0, 1]))

In [10]:
root_tmp / "input_inference.npy"

PosixPath('/neurospin/dico/tsanchez/tmp/input_inference.npy')

: 

In [5]:
a = ana.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module gltf_io
loading module palettecontrols
loading module paletteViewer
loading module meshsplit
loading module profilewindow
loading module ana_image_math
loading module anacontrolmenu
loading module foldsplit
loading module modelGraphs
loading module bsa_proba
loading module histogram
loading module gradientpalette
loading module infowindow
loading module simple_controls
loading module volumepalettes
loading module statsplotwindow
loading module save_resampled
loading module valuesplotwindow
loading module selection
all python modules loaded
Anatomist started.


In [6]:
inputs_tens = torch.from_numpy(inputs)
inputs_tens = inputs_tens.squeeze(0) 
inputs_reshaped = inputs_tens.unsqueeze(-1)
inputs_reshaped.shape

torch.Size([60, 120, 144, 1])

In [7]:
vol_inputs = aims.Volume(inputs_reshaped.numpy().astype(np.int16))
np.unique(vol_inputs.np, return_counts=True)
vol_inputs.header(), np.unique(vol_inputs.np)

({ 'volume_dimension' : [ 60, 120, 144, 1 ], 'sizeX' : 60, 'sizeY' : 120, 'sizeZ' : 144, 'sizeT' : 1, 'referential' : 'e8ada048-21a4-11f0-99ba-e43d1ab1cb49' },
 array([-1,  0,  1], dtype=int16))

In [8]:
A_inputs = a.toAObject(vol_inputs)

In [9]:
vol_3d = a.fusionObjects(
    objects=[A_inputs],
    method= "VolumeRenderingFusionMethod"
)

clipped = a.fusionObjects(
    objects=[vol_3d],
    method= "FusionClipMethod"
)

clipped.releaseAppRef()

In [10]:
colors = [0, 0, 0] * 512
palette = a.createPalette("customPal")
palette.setColors(colors) #Value of 512

palette.np["v"][100:400,:,:,:,3] = 0 # Settings middle alpha to 0
palette.np["v"][:100,:,:,:,2] = 255 # Settings begin to blue max
palette.np["v"][:100,:,:,:,1] = 255 # Settings begin to green max
palette.np["v"][400:,:,:,:,0] = 255 # Settings end to red max

# palette_array = np.zeros((512,1,1,1,4))
# palette_array[100:400,:,:,:,3] = 0 # Settings middle alpha to 0
# palette_array[:100,:,:,:,2] = 255 # Settings begin to blue max
# palette_array[:100,:,:,:,1] = 255 # Settings begin to green max
# palette_array[400:,:,:,:,0] = 255 # Settings end to red max

# palette.np["v"][:] = palette_array[:]

In [11]:
palette.np["v"].shape

(512, 1, 1, 1, 4)

In [12]:
clipped.setPalette(palette,
                  absoluteMode = True)

In [13]:
palette

In [14]:
clipped.setQuaternion(aims.Quaternion([0.6427876096865394,
  -0.3420201433256688,
  0.6427876096865394,
  0.3420201433256688]))

In [15]:
win = a.createWindow("3D")
win.setHasCursor(0)
win.addObjects(clipped)

In [16]:
clipped.getInfos()

{'children': [3],
 'copy': 0,
 'loadDate': 1745507077,
 'material': {'ambient': [0.100000001490116,
   0.100000001490116,
   0.100000001490116,
   1],
  'diffuse': [0.800000011920929,
   0.800000011920929,
   0.800000011920929,
   0.200000002980232],
  'emission': [0, 0, 0, 1],
  'face_culling': 0,
  'selectable_mode': 'selectable_when_opaque',
  'shininess': 20,
  'specular': [0.200000002980232, 0.200000002980232, 0.200000002980232, 1],
  'unlit_color': [0, 0, 0, 1]},
 'multiObject': 1,
 'name': 'ClippedObject: VolumeRendering: Volume_S16',
 'objectType': 'ClippedObject',
 'palette': {'colorMixSize': 512,
  'linMixFactor': 0.5,
  'max': 1,
  'max2': 1,
  'min': 0.25,
  'min2': 0,
  'mixMethod': 'GEOMETRIC',
  'palette': 'customPal',
  'palette1Dmapping': 'FirstLine',
  'sizex': 0,
  'sizey': 0},
 'referential': 2,
 'slice_plane': [1, 0, 0, -29.5],
 'slice_position': [29.5, 59.5, 71.5],
 'slice_quaternion': [0.642787635326385,
  -0.342020153999329,
  0.642787635326385,
  0.342020153999

In [17]:
win.getInfos()


{'boundingbox_max': [30, 72, 61.2120018005371],
 'boundingbox_min': [-30, -72, -61.2000007629395],
 'geometry': [2, 82, 424, 477],
 'group': 0,
 'objects': [0],
 'observer_position': [29.5, 59.5, 71.5],
 'position': [29.5, 59.5, 71.5, 0],
 'referential': 2,
 'selected': 0,
 'slice_quaternion': [0, 0, 0, 1],
 'type': 'AWindow',
 'view_quaternion': [0.70710676908493, 0, 0, 0.70710676908493],
 'view_size': [384, 384],
 'windowType': '3D',
 'zoom': 1}

In [20]:
quat = [0.5,-0.5,-0.5,0.5] # Sagittal view
# quat = [0.6,-0.2,-0.25,0.75] # Sagittal view

In [21]:
# quat = [-0.52179491519928, -0.373378068208694, 0.757112264633179,0.122882008552551] # Random
win.camera(view_quaternion=quat, zoom=0.67)

: 